In [1]:
import sys
sys.path.append('..')

In [2]:
import sentence_embedding_evaluation_german as seeg
import torch
from typing import List

/Users/uh/projects/paper-293/sentence-embedding-evaluation-german/.venv/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## (1) Instantiate your Embedding model
First, you should load your pretrained embedding.

Here we will generate a random embedding for demonstration purposes.

In [3]:
# generate a random embedding
emb_dim = 512
vocab_sz = 128
emb = torch.randn((vocab_sz, emb_dim), requires_grad=False)
emb = torch.nn.Embedding.from_pretrained(emb)
# assert emb.weight.requires_grad == False

## (2) Specify the preprocessing
The `preprocessor` function converts a sentences as string into embedding vectors of numbers.

Here we will convert the input strings with a nonsensical approach into IDs for the Embedding layer.

In [4]:
def preprocesser(batch: List[str], params: dict=None) -> List[List[float]]:
    """ Specify your embedding or pretrained encoder here
    Paramters:
    ----------
    params : dict
        The params dictionary
    batch : List[str]
        A list of sentence as string
    Returns:
    --------
    List[List[float]]
        A list of embedding vectors
    """
    features = []
    for sent in batch:
        try:
            ids = torch.tensor([ord(c) % 128 for c in sent])
        except:
            print(sent)
        h = emb(ids)
        features.append(h.mean(axis=0))
    features = torch.stack(features, dim=0)
    return features

## (3) Training settings

In [5]:
params = {
    'datafolder': '../datasets',
    'bias': True,
    'balanced': True,
    'batch_size': 64, 
    'num_epochs': 20,
    # 'early_stopping': True,
    # 'split_ratio': 0.2,  # if early_stopping=True
    # 'patience': 5,  # if early_stopping=True
}

## (4) Specify downstream tasks

In [6]:
# All
# downstream_tasks = [
#     'TOXIC', 'ENGAGE', 'FCLAIM', 'VMWE',
#     'OL19-A', 'OL19-B', 'OL19-C',
#     'OL18-A', 'OL18-B', 
#     'ABSD-1', 'ABSD-2', 'ABSD-3',
#     'MIO-S', 'MIO-O', 'MIO-I', 'MIO-D', 'MIO-F', 'MIO-P', 'MIO-A',
#     'SBCH-L', 'SBCH-S', 'ARCHI', 'LSDC'
# ]

# Group tasks
# downstream_tasks = [
#     'ABSD-2', 'MIO-S', 'SBCH-S',  # Sentiment analysis
#     'ENGAGE', 'MIO-P',  # engaging/personal
#     'FCLAIM', 'MIO-A',  # fact-claim (potential fake news), argumentative, reasoning
#     'TOXIC', 'OL19-A', 'OL19-B', 'OL19-C', 'MIO-O', 'MIO-I',  # toxic
# ]

# Current favorites
downstream_tasks = ['FCLAIM', 'VMWE', 'OL19-C', 'ABSD-2', 'MIO-P', 'ARCHI', 'LSDC']

## (5) Run experiments

In [7]:
%%time
results = seeg.evaluate(downstream_tasks, preprocesser, **params)

CPU times: user 2min 1s, sys: 40.4 s, total: 2min 42s
Wall time: 3min 5s


## (6) Display results

In [8]:
print("Task | Epochs | N train | N test")
for res in results:
    print(f"{res['task']:>7s}: {res['epochs']:5d} {res['train']['num']:6d} {res['test']['num']:6d}")

Task | Epochs | N train | N test
 FCLAIM:    20   3244    944
   VMWE:    20   6652   1447
 OL19-C:    20   1921    930
 ABSD-2:    20  19432   2555
  MIO-P:    20   4668   4668
  ARCHI:    20  18809   4743
   LSDC:    20  74140   8602


In [9]:
metric = 'f1-balanced'
print('  Task | train | test')
for res in results:
    print(f"{res['task']:>7s}: {res['train'][metric]:6.3f} {res['test'][metric]:6.3f}")

  Task | train | test
 FCLAIM:  0.596  0.606
   VMWE:  0.453  0.451
 OL19-C:  0.570  0.523
 ABSD-2:  0.406  0.367
  MIO-P:  0.581  0.587
  ARCHI:  0.401  0.395
   LSDC:  0.321  0.302


In [10]:
# class label distributions (inference)
for res in results:
    print(res['task'], res['test']['num'])
    print(res['test']['distr-test'])
    print(res['test']['distr-pred'])

FCLAIM 944
{0: 630, 1: 314}
{0: 579, 1: 365}
VMWE 1447
{1: 256, 0: 1191}
{0: 1447}
OL19-C 930
{0: 796, 1: 134}
{0: 731, 1: 199}
ABSD-2 2555
{1: 1670, 0: 780, 2: 105}
{0: 796, 1: 1217, 2: 542}
MIO-P 4668
{1: 812, 0: 3856}
{1: 2099, 0: 2569}
ARCHI 4743
{2: 1177, 3: 1176, 1: 1199, 0: 1191}
{2: 826, 0: 811, 3: 1457, 1: 1649}
LSDC 8602
{13: 280, 6: 346, 12: 940, 3: 925, 2: 944, 1: 934, 11: 931, 0: 453, 10: 511, 5: 924, 4: 65, 8: 923, 9: 83, 7: 343}
{5: 1208, 1: 1084, 8: 1198, 11: 1176, 12: 997, 2: 567, 3: 1261, 10: 1111}


In [11]:
# class label distribution (training)
for res in results:
    print(res['task'], res['train']['num'])
    print(res['train']['distr-train'])
    print(res['train']['distr-pred'])

FCLAIM 3244
{0: 2141, 1: 1103}
{1: 1286, 0: 1958}
VMWE 6652
{1: 1145, 0: 5507}
{0: 6650, 1: 2}
OL19-C 1921
{0: 1664, 1: 257}
{0: 1519, 1: 402}
ABSD-2 19432
{2: 1179, 1: 13208, 0: 5045}
{2: 4451, 0: 6136, 1: 8845}
MIO-P 4668
{0: 3855, 1: 813}
{0: 2482, 1: 2186}
ARCHI 18809
{1: 4797, 3: 4407, 2: 4802, 0: 4803}
{1: 7483, 2: 3619, 0: 2869, 3: 4838}
LSDC 74140
{12: 15019, 8: 7829, 5: 13506, 1: 5294, 11: 13227, 3: 11002, 2: 5704, 13: 346, 10: 749, 7: 382, 9: 143, 0: 469, 6: 377, 4: 93}
{2: 5176, 8: 10234, 1: 5518, 11: 14064, 5: 13096, 3: 11011, 12: 11158, 10: 3773, 7: 100, 13: 9, 6: 1}
